In [1]:
import numpy as np
import pandas as pd
import folium
import branca
from matplotlib import pyplot as plt
from collections import Counter
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

SEED = 25

In [2]:
df = pd.concat([pd.read_csv('train.csv'),pd.read_csv('test.csv')])

In [3]:
df.head()

,Unnamed: 0,id,atm_group,address,address_rus,lat,long,target
0,0,8526.0,32.0,"EMELYANOVA,34 Y-SAKHALINSK","улица А.О. Емельянова, 34, Южно-Сахалинск, Сах...",46.940995,142.738319,0.011500
1,1,8532.0,32.0,"KOMSOMOLSKAYA,259B Y.SAKHALINSK","Комсомольская улица, 259, Южно-Сахалинск, Саха...",46.937353,142.753348,0.029710
2,2,8533.0,32.0,"KOMMUN. PR., 32 YUZHNO SAKHAL","Коммунистический проспект, Южно-Сахалинск, Сах...",46.959413,142.741113,0.009540
3,3,8684.0,32.0,"LENINGRADSKIY PR.,76A MOSCOW","Ленинградский проспект, 76А, Москва, Россия, 1...",55.805827,37.515146,-0.094035
4,4,37.0,32.0,"GVARDEYSKAYA PL., 2 NORILSK","Гвардейская площадь, 2, Норильск, Красноярский...",69.343541,88.211228,0.079277


In [4]:
df = df.drop('Unnamed: 0', axis = 1)
df.head()

,id,atm_group,address,address_rus,lat,long,target
0,8526.0,32.0,"EMELYANOVA,34 Y-SAKHALINSK","улица А.О. Емельянова, 34, Южно-Сахалинск, Сах...",46.940995,142.738319,0.011500
1,8532.0,32.0,"KOMSOMOLSKAYA,259B Y.SAKHALINSK","Комсомольская улица, 259, Южно-Сахалинск, Саха...",46.937353,142.753348,0.029710
2,8533.0,32.0,"KOMMUN. PR., 32 YUZHNO SAKHAL","Коммунистический проспект, Южно-Сахалинск, Сах...",46.959413,142.741113,0.009540
3,8684.0,32.0,"LENINGRADSKIY PR.,76A MOSCOW","Ленинградский проспект, 76А, Москва, Россия, 1...",55.805827,37.515146,-0.094035
4,37.0,32.0,"GVARDEYSKAYA PL., 2 NORILSK","Гвардейская площадь, 2, Норильск, Красноярский...",69.343541,88.211228,0.079277


Изучить особенности данных:
1)Рассчитать основные статистики для вещественных и категориальных признаков (df.describe(include=’all’))


In [5]:
df.describe(include='all')

,id,atm_group,address,address_rus,lat,long,target
count,8765.000000,8765.000000,8765,8345,8345.000000,8345.000000,6261.000000
unique,NaN,NaN,7481,6147,NaN,NaN,NaN
top,NaN,NaN,"HOROSHEVSKOE SH,V38 MOSKVA","квартал В-13, городской округ Волгодонск, Рост...",NaN,NaN,NaN
freq,NaN,NaN,7,10,NaN,NaN,NaN
mean,4404.586366,4407.008842,NaN,NaN,54.138304,63.500873,0.000715
std,2542.532548,2427.934099,NaN,NaN,4.329293,32.787366,0.085852
min,1.000000,32.000000,NaN,NaN,42.057080,19.941530,-0.145001
25%,2207.000000,1942.000000,NaN,NaN,52.243997,37.851107,-0.061176
50%,4403.000000,5478.000000,NaN,NaN,55.060936,49.071694,-0.015573
75%,6603.000000,5478.000000,NaN,NaN,56.116673,84.950218,0.040149


Есть ли пропущенные значения. Сколько. Какова их доля от общего числа объектов для каждого конкретного признака. И, в целом, насколько ваш датасет “разрежен” в контексте пропущенных значений. Выдвинуть и (желательно) проверить гипотезу, что могут означать пропущенные значения? Осмысленно и аргументированно обработать пропуски.

In [6]:
df[df.isnull().sum(axis = 1) == True]

,id,atm_group,address,address_rus,lat,long,target
0,8096.0,5478.0,"D. 97, UL. KIROVA NOVOKUZNETSK","улица Кирова, 97, Новокузнецк, Кемеровская обл...",53.757824,87.159786,NaN
2,2428.0,3185.5,"UL.NAHIMOVA, DOM 33 VILJUCHINSK","улица Нахимова, 33, жилой район Рыбачий, Вилюч...",52.918598,158.514038,NaN
3,2799.0,3185.5,MKR 3 D 13B SHAR SHARYPOVO,"3-й микрорайон, Шарыпово, Красноярский край, Р...",55.533496,89.178148,NaN
4,250.0,1942.0,29 CHICHERINA STR. CHELYABINSK,"улица Чичерина, 29, Челябинск, Россия, 454128",55.179216,61.297729,NaN
5,5759.0,5478.0,"D. 162, UL. BOGDANA HMEL' OMSK G","улица Богдана Хмельницкого, 162/100, Омск, Рос...",54.971827,73.411646,NaN
...,...,...,...,...,...,...,...
2499,7805.0,5478.0,"D. 12, PROEZD ZAREVYI MOSKVA G","Заревый проезд, 12, Москва, Россия, 127282",55.886488,37.656829,NaN
2500,7395.0,5478.0,"D. 2B, UL. MIRA SOCHI G","улица Мира, 4, Адлерский район, Сочи, Краснода...",43.438310,39.912211,NaN
2501,3339.0,8083.0,"KARMANITSKIY, 9 Moskva","Карманицкий переулок, 9, Москва, Россия, 119002",55.748275,37.583571,NaN
2502,675.0,1942.0,3A SHARAPOVSKIY WAY MYTISHCHI,"Шараповская улица, 1, Мытищи, Московская облас...",55.910992,37.748466,NaN


In [7]:
#не убрал target из выборки

In [8]:
y = df['target']
X = df.drop('target', axis = 1)

Есть ли ошибочные  (например, отрицательный возраст или пятиметровый рост человека; 3-й класс в задаче бинарной классификации) значения (признаки, целевая переменная) в данных. Обработать их.

In [9]:
X.describe(include='all')

,id,atm_group,address,address_rus,lat,long
count,8765.000000,8765.000000,8765,8345,8345.000000,8345.000000
unique,NaN,NaN,7481,6147,NaN,NaN
top,NaN,NaN,"HOROSHEVSKOE SH,V38 MOSKVA","квартал В-13, городской округ Волгодонск, Рост...",NaN,NaN
freq,NaN,NaN,7,10,NaN,NaN
mean,4404.586366,4407.008842,NaN,NaN,54.138304,63.500873
std,2542.532548,2427.934099,NaN,NaN,4.329293,32.787366
min,1.000000,32.000000,NaN,NaN,42.057080,19.941530
25%,2207.000000,1942.000000,NaN,NaN,52.243997,37.851107
50%,4403.000000,5478.000000,NaN,NaN,55.060936,49.071694
75%,6603.000000,5478.000000,NaN,NaN,56.116673,84.950218


In [10]:
#id совпадает с ожиданиями (от 1 до 8765)

In [11]:
Counter(X['atm_group'])

Counter({32.0: 62,
         496.5: 863,
         1022.0: 185,
         1942.0: 1646,
         3185.5: 826,
         5478.0: 3729,
         8083.0: 1454})

In [12]:
#групп банкоматов всего 7

In [13]:
#адреса и столбцы координат содержат пропуски, про это ниже

In [14]:
Counter(y)

Counter({0.0114998209182048: 1,
         0.0297102650350859: 1,
         0.0095399608832554: 1,
         -0.0940346244764994: 1,
         0.0792773520096629: 1,
         -0.1450012314747548: 55,
         -0.0246822352357643: 2,
         -0.1334258834962322: 2,
         0.1414693533331615: 1,
         0.0172589332663376: 1,
         -0.0139490047116537: 1,
         -0.0484357394087966: 2,
         -0.0642932974491817: 1,
         -0.1001214289644553: 4,
         -0.0008294956654186: 1,
         -0.0476678404207127: 1,
         0.017372643984509: 1,
         -0.0276573657875183: 1,
         -0.0778711918613832: 1,
         -0.0506266855287384: 2,
         0.0190607428520956: 1,
         -0.0274751961229577: 1,
         -0.1013755908540726: 2,
         -0.086529509128304: 1,
         -0.0732875290072723: 1,
         -0.1105973682659273: 1,
         -0.0787939986779948: 1,
         -0.0261465882319446: 1,
         0.0182088275363533: 1,
         0.0613509276698259: 1,
         -0.044730021

In [15]:
X.isnull().sum().sum()

1260

In [16]:
X[X.isnull().sum(axis = 1) > 1]

,id,atm_group,address,address_rus,lat,long
40,4510.0,496.5,PER. BAZARNY 4 SPASSK-DALNY,NaN,NaN,NaN
150,347.0,1022.0,ABB 6B NAB-CHELNINSKIJ NAB.CHELNY,NaN,NaN,NaN
182,1591.0,1942.0,16A BEREJKOVSK NAB MOSCOW,NaN,NaN,NaN
192,440.0,1942.0,113 MIRA AV. TOGLIATTI,NaN,NaN,NaN
214,556.0,1942.0,133A MOZHAYSKOE H/W. MOSCOW,NaN,NaN,NaN
...,...,...,...,...,...,...
2346,3199.0,8083.0,"MOROZOVA P.L., 86 KHabarovsk",NaN,NaN,NaN
2399,5354.0,5478.0,"D. 15, UL. 78 DOBROVOL'CH KRASNOYARSK",NaN,NaN,NaN
2471,3333.0,8083.0,"OKTYABRSKAYA, 105 Tver",NaN,NaN,NaN
2474,2247.0,3185.5,"UL. TELEVIZORNAJA, KRASNOJARSK",NaN,NaN,NaN


In [17]:
#у нас есть целых 420 строк, где нет адреса кириллицей, но есть латиницей
#Также в этих строках нет координат
#сделать вывод повлияют ли это строки на работу модели сложно, поэтому лучше их заполнить

In [18]:
import requests

def translate(s):
    key = "trnsl.1.1.20141031T091526Z.c7258ad3549a5c3b.4014cc4efaf6aa8bba976eb04da8f906b44d4076"
    r = requests.get('https://translate.yandex.net/api/v1.5/tr.json/translate?key={}&text={}&lang=en-ru'.format(key, s))
    r = r.json()
    if r["code"] == 200:
        return r["text"][0]
    else:
        return None

In [ ]:
translated  = X[X.isnull().sum(axis = 1) > 1]['address'].apply(translate)

In [ ]:
X['address_rus'][X[X.isnull().sum(axis = 1) > 1]] =

In [ ]:
def geocode(s):
    apikey = "6c46c2d4-efa9-4638-b868-f32a87cd9066"
    r = requests.get('https://geocode-maps.yandex.ru/1.x/?format=json&apikey={}&geocode={}'.format(apikey, s))
    r = r.json()
    if int(r["response"]["GeoObjectCollection"]["metaDataProperty"]["GeocoderResponseMetaData"]["found"]) > 0:
        return r["response"]["GeoObjectCollection"]["featureMember"][0]["GeoObject"]["Point"]["pos"].split()
    else:
        return np.NaN, np.NaN

In [ ]:
df.target.hist();

In [ ]:
fmap = folium.Map(location=[55.805827, 70.515146], max_zoom=7)
colorscale = branca.colormap.linear.RdBu_04.scale(df.target.min(), df.target.max())
for index, row in df[["lat", "long", "isTrain", "target"]].dropna(subset=["lat", "long"]).sample(frac=1).iterrows():
    if row.isTrain:
        folium.CircleMarker(location=[row["lat"], row["long"]],
                            radius=10,
                            opacity=1,
                            weight=3,
                            color=colorscale(row.target),
                            tooltip=f"Index:{index},\nTarget: {row.target}").add_to(fmap)
    else:
        folium.RegularPolygonMarker(location=[row["lat"], row["long"]],
                            radius=10,
                            fill_opacity=0,
                            opacity=0.7,
                            weight=3,
                            color="#999999",
                            number_of_sides=3,
                            tooltip=index).add_to(fmap)

In [ ]:
fmap.save('rosbank.html')